# Assignment 18: Time Series Analysis

## Dataset: Exchange Rate Data

**Topics Covered:**
- Time Series Decomposition
- Stationarity Testing
- ARIMA Forecasting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Load data
df = pd.read_csv('exchange_rate.csv')
print("Dataset loaded! Shape:", df.shape)
df.head()

In [ ]:
# Parse dates and set index
date_col = df.columns[0]
df[date_col] = pd.to_datetime(df[date_col])
df.set_index(date_col, inplace=True)

# Select the value column
value_col = df.select_dtypes(include=[np.number]).columns[0]
ts = df[value_col]

print("Time series ready!")
print("Date range:", ts.index.min(), "to", ts.index.max())

In [ ]:
# Plot time series
plt.figure(figsize=(14, 5))
plt.plot(ts)
plt.title('Exchange Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.grid(True)
plt.savefig('time_series.png')
plt.show()

In [ ]:
# Decomposition
print("=== Time Series Decomposition ===")

# Use a subset for decomposition if data is too large
decomposition = seasonal_decompose(ts.dropna(), model='additive', period=30)

fig, axes = plt.subplots(4, 1, figsize=(14, 12))
decomposition.observed.plot(ax=axes[0], title='Original')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')
plt.tight_layout()
plt.savefig('decomposition.png')
plt.show()

In [ ]:
# Stationarity Test (ADF Test)
print("=== ADF Test for Stationarity ===")

result = adfuller(ts.dropna())
print("ADF Statistic:", round(result[0], 4))
print("p-value:", round(result[1], 4))

if result[1] < 0.05:
    print("Result: Series is STATIONARY")
else:
    print("Result: Series is NON-STATIONARY")

In [ ]:
# ACF and PACF plots
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(ts.dropna(), ax=axes[0], lags=40)
plot_pacf(ts.dropna(), ax=axes[1], lags=40)
plt.tight_layout()
plt.savefig('acf_pacf.png')
plt.show()

In [ ]:
# ARIMA Model
print("=== ARIMA Model ===")

# Use a subset for faster training
ts_subset = ts.dropna()[-500:]

# Split into train/test
train_size = int(len(ts_subset) * 0.8)
train = ts_subset[:train_size]
test = ts_subset[train_size:]

# Fit ARIMA
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Forecast
forecast = model_fit.forecast(steps=len(test))

plt.figure(figsize=(14, 5))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast, label='Forecast', color='red')
plt.legend()
plt.title('ARIMA Forecast')
plt.savefig('arima_forecast.png')
plt.show()

## Summary

- Decomposed time series into trend, seasonal, and residual
- Tested for stationarity using ADF test
- Built ARIMA model for forecasting